In [12]:
%%writefile radio.py

import json
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
import requests
from datetime import date, datetime
import plotly.express as px

# 데이터 로딩
data = pd.read_excel("stream_data.xlsx")

# 스트림릿 상태 초기화
if 'start_date' not in st.session_state:
    st.session_state.start_date = datetime(2023, 10, 1)
if 'end_date' not in st.session_state:
    st.session_state.end_date = datetime(2023, 11, 1)

# 사이드바에 필터 추가
st.sidebar.header("Filters")
start_date = st.sidebar.date_input("Start date", st.session_state.start_date, min_value=date(2023, 10, 1))
end_date = st.sidebar.date_input("End date", st.session_state.end_date, max_value=date(2023, 11, 1))

# datetime 형식으로 변환
start_date = datetime.combine(start_date, datetime.min.time())
end_date = datetime.combine(end_date, datetime.min.time())

# st.session_state 변수 업데이트
st.session_state.start_date = start_date
st.session_state.end_date = end_date

# 선택한 날짜에 따라 데이터프레임 필터링
filtered_data = data[
    (data['일자'] >= start_date) & (data['일자'] <= end_date)
]

new_genre_list = st.sidebar.multiselect('Choose topics:', data['topics'].unique(), default=['전쟁'])
# 한국 도시와 외국 도시 데이터를 나누어 저장
korean_data = data[data['외국유무'] == '한국']
foreign_data = data[data['외국유무'] == '외국']

# tabs 만들기
tab1, tab2 = st.tabs(["국내", "해외"])
with tab1:
    col1, col2 = st.columns([5, 5])
    # 데이터프레임 필터링
    filtered_korean_data = data[
        (data['일자'] >= start_date) & (data['일자'] <= end_date) & (data['외국유무'] == '한국')
    ]    
    with col1:
        topics_distribution = filtered_korean_data['topics'].value_counts().reset_index()
        fig_topics = px.pie(
            topics_distribution,
            names='index',
            values='topics',
            title='한국 Topics 분포',
            hole=0.4,  # 도넛 차트 설정
            color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
        )
        fig_topics.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True,  # 범례 표시
            legend_title="Topics" 
        )
        st.plotly_chart(fig_topics)

# 가운데 열에는 한국 도시별 리스크 그래프 표시
    with col2:
        korean_top_5 = filtered_korean_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
        fig_korean = px.bar(
            korean_top_5,
            y='risk',
            x='city',
            orientation='v',
            title="Top 5 한국 도시별 리스크",
            color='risk',  # 색상을 리스크에 따라 설정
            color_continuous_scale='viridis'  # 색상 맵 설정
        )
        fig_korean.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True
        )
        st.plotly_chart(fig_korean)
        
        # 상위 2개 도시에 해당하는 데이터프레임 출력
        top_2_cities_data = filtered_korean_data[filtered_korean_data['city'].isin(korean_top_5['city'].head(2))]
        st.write("Top 2 Cities Data:")
        st.dataframe(top_2_cities_data[['일자', 'city', '제목', '요약']])
        

# 해외 탭에 대한 그래프 추가
with tab2:
    col1, col2 = st.columns([5, 5])
    
    # 데이터프레임 필터링
    filtered_foreign_data = data[
        (data['일자'] >= start_date) & (data['일자'] <= end_date) & (data['외국유무'] == '외국')
    ]
    
    with col1:
        topics_distribution_foreign = filtered_foreign_data['topics'].value_counts().reset_index()
        fig_topics_foreign = px.pie(
            topics_distribution_foreign,
            names='index',
            values='topics',
            title='해외 Topics 분포',
            hole=0.4,
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig_topics_foreign.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True,
            legend_title="Topics" 
        )
        st.plotly_chart(fig_topics_foreign)

    # 가운데 열에는 해외 도시별 리스크 그래프 표시
    with col2:
        foreign_top_5 = filtered_foreign_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
        fig_foreign = px.bar(
            foreign_top_5,
            y='risk',
            x='city',
            orientation='v',
            title="Top 5 해외 도시별 리스크",
            color='risk',
            color_continuous_scale='viridis'
        )
        fig_foreign.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True
        )
        st.plotly_chart(fig_foreign)
        
        # 상위 2개 도시에 해당하는 데이터프레임 출력
        top_2_foreign_cities_data = filtered_foreign_data[filtered_foreign_data['city'].isin(foreign_top_5['city'].head(2))]
        st.write("Top 2 Foreign Cities Data:")
        st.dataframe(top_2_foreign_cities_data[['일자', 'city', '제목', '요약']])

Overwriting radio.py


In [14]:
%%writefile radio.py

import json
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
import requests
from datetime import date, datetime
import plotly.express as px

# 데이터 로딩
data = pd.read_excel("stream_data.xlsx")

# 스트림릿 상태 초기화
if 'start_date' not in st.session_state:
    st.session_state.start_date = datetime(2023, 10, 1)
if 'end_date' not in st.session_state:
    st.session_state.end_date = datetime(2023, 11, 1)

# 사이드바에 필터 추가
st.sidebar.header("Filters")
start_date = st.sidebar.date_input("Start date", st.session_state.start_date, min_value=date(2023, 10, 1))
end_date = st.sidebar.date_input("End date", st.session_state.end_date, max_value=date(2023, 11, 1))

# datetime 형식으로 변환
start_date = datetime.combine(start_date, datetime.min.time())
end_date = datetime.combine(end_date, datetime.min.time())

# st.session_state 변수 업데이트
st.session_state.start_date = start_date
st.session_state.end_date = end_date

# 선택한 날짜에 따라 데이터프레임 필터링
filtered_data = data[
    (data['일자'] >= start_date) & (data['일자'] <= end_date)
]

new_genre_list = st.sidebar.multiselect('Choose topics:', data['topics'].unique(), default=['전쟁'])
# 한국 도시와 외국 도시 데이터를 나누어 저장
korean_data = data[data['외국유무'] == '한국']
foreign_data = data[data['외국유무'] == '외국']

# tabs 만들기
tab1, tab2 = st.tabs(["국내", "해외"])
with tab1:
    col1, col2 = st.columns([5, 5])
    # 데이터프레임 필터링
    filtered_korean_data = data[
        (data['일자'] >= start_date) & (data['일자'] <= end_date) & (data['외국유무'] == '한국')
    ]    
    with col1:
        topics_distribution = filtered_korean_data['topics'].value_counts().reset_index()
        fig_topics = px.pie(
            topics_distribution,
            names='index',
            values='topics',
            title='한국 Topics 분포',
            hole=0.4,  # 도넛 차트 설정
            color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
        )
        fig_topics.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True,  # 범례 표시
            legend_title="Topics" 
        )
        st.plotly_chart(fig_topics)

# 가운데 열에는 한국 도시별 리스크 그래프 표시
    with col2:
        korean_top_5 = filtered_korean_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
        fig_korean = px.bar(
            korean_top_5,
            y='risk',
            x='city',
            orientation='v',
            title="Top 5 한국 도시별 리스크",
            color='risk',  # 색상을 리스크에 따라 설정
            color_continuous_scale='viridis'  # 색상 맵 설정
        )
        fig_korean.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True
        )
        st.plotly_chart(fig_korean)
        
        # 상위 2개 도시에 해당하는 데이터프레임 출력
        selected_city_radio = st.radio("Select City", korean_top_5['city'].head(2))
        top_2_cities_data = filtered_korean_data[filtered_korean_data['city'] == selected_city_radio]
        st.write(f"Selected City Data for {selected_city_radio}:")
        st.dataframe(top_2_cities_data[['일자', 'city', '제목', '요약']])
        

# 해외 탭에 대한 그래프 추가
with tab2:
    col1, col2 = st.columns([5, 5])
    
    # 데이터프레임 필터링
    filtered_foreign_data = data[
        (data['일자'] >= start_date) & (data['일자'] <= end_date) & (data['외국유무'] == '외국')
    ]
    
    with col1:
        topics_distribution_foreign = filtered_foreign_data['topics'].value_counts().reset_index()
        fig_topics_foreign = px.pie(
            topics_distribution_foreign,
            names='index',
            values='topics',
            title='해외 Topics 분포',
            hole=0.4,
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig_topics_foreign.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True,
            legend_title="Topics" 
        )
        st.plotly_chart(fig_topics_foreign)

    # 가운데 열에는 해외 도시별 리스크 그래프 표시
    with col2:
        foreign_top_5 = filtered_foreign_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
        fig_foreign = px.bar(
            foreign_top_5,
            y='risk',
            x='city',
            orientation='v',
            title="Top 5 해외 도시별 리스크",
            color='risk',
            color_continuous_scale='viridis'
        )
        fig_foreign.update_layout(
            width=400,
            height=400,
            title_x=0.5,
            margin=dict(l=0, r=0, t=30, b=0),
            showlegend=True
        )
        st.plotly_chart(fig_foreign)
        
        # 상위 2개 도시에 해당하는 데이터프레임 출력
        selected_city_radio_foreign = st.radio("Select City", foreign_top_5['city'].head(2))
        top_2_foreign_cities_data = filtered_foreign_data[filtered_foreign_data['city'] == selected_city_radio_foreign]
        st.write(f"Selected City Data for {selected_city_radio_foreign}:")
        st.dataframe(top_2_foreign_cities_data[['일자', 'city', '제목', '요약']])

Overwriting radio.py
